In [1]:
from datetime import datetime
import requests
import pandas as pd
import numpy as np
import pickle
import json

In [21]:
class weather_request:
    
    def __init__(self, lat, long, t_start, n_days, timezone):
        self.lat = lat
        self.long = long
        self.n_days = n_days
        self.daysecond = 3600*24
        self.requests_list = []
        self.json_objs = []
        self.timezone = timezone
        
        if(isinstance(t_start, str)):
            datetime_object = datetime.strptime(t_start, "%Y-%m-%d %H:%M")
            
            self.t_start = int(datetime_object.timestamp())
        else:
            raise TypeError('fill in time format "YYYY-MM-DD HH:MM".')
             
        
        
    def request(self):
        for i in range(self.n_days):
            print()
            req = requests.get("https://api.darksky.net/forecast/9ae4fb3bd1f83bc1076d8f367b3d27b4/{},{},{}?units=si"
                                   .format(self.lat, self.long, self.t_start + i*self.daysecond))
            self.requests_list.append(req)
        
    def get_pandas(self):
        if len(self.requests_list) == 0:
            raise Exception("user should do the request() first")
        else:
            
            for js_obj in self.requests_list:
                self.json_objs += js_obj.json()['hourly']['data']
            with open('temp.json', 'w') as outfile:
                json.dump(self.json_objs, outfile)
            jspd = pd.read_json('temp.json',orient='columns')
            ## data frame cleaning
            jspd['time'] = pd.to_datetime(jspd['time'], unit='s')
            jspd['time'] = jspd['time'].dt.tz_localize("UTC").dt.tz_convert(self.timezone)
            jspd['time'] = jspd['time'].apply(lambda x:datetime.replace(x,tzinfo=None))
            jspd.set_index(jspd['time'], inplace = True)
            jspd.drop(columns = ['time'], inplace = True)
            return jspd
        return 

In [17]:
wr = weather_request(50.8436, 4.3530, "2020-01-01 00:00", 5, 'CET')

In [18]:
wr.request()

In [14]:
wr.json_objs

[]

In [19]:
df = wr.get_pandas()

In [20]:
df.head()

,summary,icon,precipIntensity,precipProbability,temperature,apparentTemperature,dewPoint,humidity,pressure,windSpeed,windGust,windBearing,cloudCover,uvIndex,visibility,ozone,precipType
time,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00,Partly Cloudy,partly-cloudy-night,0.0,0.0,34.63,29.48,29.39,0.81,1034.0,5.84,9.38,117,0.35,0,10,309.9,NaN
2020-01-01 01:00:00,Clear,clear-night,0.0,0.0,33.91,29.01,28.59,0.81,1033.8,5.35,7.73,129,0.27,0,10,307.2,NaN
2020-01-01 02:00:00,Clear,clear-night,0.0,0.0,35.57,30.62,30.52,0.82,1033.6,5.81,7.59,115,0.19,0,10,305.8,NaN
2020-01-01 03:00:00,Clear,clear-night,0.0,0.0,34.22,29.30,29.09,0.81,1032.9,5.45,7.60,126,0.09,0,10,306.6,NaN
2020-01-01 04:00:00,Clear,clear-night,0.0,0.0,33.03,28.22,27.78,0.81,1032.2,5.07,7.52,135,0.02,0,10,306.6,NaN
